In [1]:
from pyspark.sql import SparkSession
from delta import *

In [2]:
import importlib_metadata


delta_version = importlib_metadata.version("delta_spark")

In [3]:
delta_version

'3.2.0'

In [4]:
# spark = (SparkSession.builder
# .appName("SparkSample")
# .master("spark://172.25.0.3:7077")
# .getOrCreate())

In [5]:
# spark = SparkSession \
#   .builder \
#   .appName("SparkSample") \
#   .master("spark://172.19.0.3:7077") \
#   .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
#   .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
#   .getOrCreate()

In [1]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [6]:
minio_bucket = 'data'
file_name = 'rejected_2007_to_2018Q4'

In [7]:
sc = spark.sparkContext

sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", "2vJ04eFHuKSANYzNIUgJ")
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "sxKh0IeLMOdf0id3X3rsamcpKFM3oJiaTpQmz9OP")
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://172.19.0.2:9000")
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")
sc._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")

# sc._jsc.hadoopConfiguration().set("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension")
# sc._jsc.hadoopConfiguration().set("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog")


In [8]:
df = spark.read.options(header=True).csv(f'./data/{file_name}.csv')

In [9]:
df.show(10)

+----------------+----------------+--------------------+----------+--------------------+--------+-----+-----------------+-----------+
|Amount Requested|Application Date|          Loan Title|Risk_Score|Debt-To-Income Ratio|Zip Code|State|Employment Length|Policy Code|
+----------------+----------------+--------------------+----------+--------------------+--------+-----+-----------------+-----------+
|          1000.0|      2007-05-26|Wedding Covered b...|     693.0|                 10%|   481xx|   NM|          4 years|        0.0|
|          1000.0|      2007-05-26|  Consolidating Debt|     703.0|                 10%|   010xx|   MA|         < 1 year|        0.0|
|         11000.0|      2007-05-27|Want to consolida...|     715.0|                 10%|   212xx|   MD|           1 year|        0.0|
|          6000.0|      2007-05-27|             waksman|     698.0|              38.64%|   017xx|   MA|         < 1 year|        0.0|
|          1500.0|      2007-05-27|              mdrigo|     5

In [ ]:
# df.write.mode("overwrite").saveAsTable("main.default.people_10m")

In [ ]:
# df.write.mode("overwrite").parquet(f"s3a://{minio_bucket}/file.parquet")

In [11]:
df.write.format("delta").save(f"s3a://{minio_bucket}/{file_name}")

Py4JJavaError: An error occurred while calling o48.save.
: java.lang.NoClassDefFoundError: org/apache/spark/sql/execution/datasources/FileFormatWriter$Empty2Null
	at org.apache.spark.sql.delta.DeltaLog.startTransaction(DeltaLog.scala:218)
	at org.apache.spark.sql.delta.DeltaLog.startTransaction(DeltaLog.scala:215)
	at org.apache.spark.sql.delta.DeltaLog.withNewTransaction(DeltaLog.scala:231)
	at org.apache.spark.sql.delta.commands.WriteIntoDelta.run(WriteIntoDelta.scala:92)
	at org.apache.spark.sql.delta.sources.DeltaDataSource.createRelation(DeltaDataSource.scala:171)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:48)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:75)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:73)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.executeCollect(commands.scala:84)
	at o

In [ ]:
# df.write.format('delta').save(f"/{file_name}")

In [ ]:
# df.write.mode("overwrite").parquet(f"s3a://{minio_bucket}/file.parquet")